# Notebook responsável por alguns aspectos de modelagem OneToOne / OneToFew Embedding (usando MongoEngine)

## Bibliotecas importantes

In [1]:
from mongoengine import *
import pprint
import numpy as np
import pandas as pd
import unidecode

# para geração de dados fake
from faker import Faker


## Classes úteis

In [2]:
class Endereco (EmbeddedDocument):
    logradouro = StringField(max_length=200, required=True)
    numero = StringField(max_length=10, required=True)
    bairro = StringField(max_length=50, required=True)
    cidade = StringField(max_length=200, required=True)
    uf = StringField(max_length=50, required=True)
    cep = StringField(max_length=10, required=True)
    
    def __str__(self):
        return f"{self.logradouro}, {self.numero}"
    
    
class Pessoa (Document):
    cpf = StringField(max_length=20, required=True)
    primeiro_nome = StringField(max_length=200, required=True)
    ultimo_nome = StringField(max_length=400, required=True)
    idade = IntField(min_value=0, max_value=120, required=True)
    data_nascimento = StringField(max_length=20, required=True)
    email = EmailField(max_length=50)
    telefones = ListField(StringField(max_length=50), max_length=4)
    formacoes = ListField(StringField(max_length=50), max_length=4)
    
    # Referência (linking) para um objeto da classe pessoa
    enderecos = ListField(EmbeddedDocumentField(Endereco))
    
    def __str__(self):
        return f"{self.pk}, {self.cpf}, {self.primeiro_nome}, Endereço: {self.enderecos}"


## Funções uteis

In [3]:
def criar_endereco():
    
    logradouro = fake.street_name()
    numero = fake.building_number()
    bairro = fake.bairro()
    cidade = fake.city()
    uf = fake.administrative_unit()
    cep = fake.postcode()

    endereco = Endereco(logradouro=logradouro, numero=numero, bairro=bairro, cidade=cidade, uf=uf, cep=cep)
    return endereco

In [4]:
def criar_pessoa():
    
    cpf = fake.cpf()
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    idade = np.random.randint(18, 50)
    data_nascimento = fake.date()
    email = f'{cpf}@{fake.free_email_domain()}'
    
    # criando os telefones
    telefones = []
    n = np.random.randint(1, 4)
    for i in range(n):
        telefones.append(fake.msisdn())
    
    # criando as formações
    formacoes = []
    n = np.random.randint(1, 4)
    for i in range(n):
        formacoes.append(fake.job())
    
    enderecos = []
    n = np.random.randint(1, 4)
    for i in range(n):
        enderecos.append(criar_endereco())
    
    
    pessoa = Pessoa(cpf=cpf, primeiro_nome=primeiro_nome, 
                    ultimo_nome=ultimo_nome, idade=idade, 
                    data_nascimento=data_nascimento, 
                    email=email, telefones=telefones, 
                    formacoes=formacoes, enderecos=enderecos)
    return pessoa

## Configurações

In [5]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de vários documentos

### Conectando ao MongoDB

In [6]:
connect('onetomany_embedding')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### Exemplo - Criação de vários documentos compostos

In [7]:
lista_pessoas = []

for i in range(10):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

[<Pessoa: None, 527.946.803-74, Rafael, Endereço: [<Endereco: Lago das Neves, 21>, <Endereco: Condomínio Ana Clara da Cunha, 11>]>,
 <Pessoa: None, 290.814.356-98, Yuri, Endereço: [<Endereco: Vale Moraes, 721>, <Endereco: Via de Dias, 114>, <Endereco: Largo Lima, 27>]>,
 <Pessoa: None, 056.273.149-07, Maria Sophia, Endereço: [<Endereco: Vereda Azevedo, 875>, <Endereco: Vereda de Viana, 82>]>,
 <Pessoa: None, 840.657.129-20, Lucas, Endereço: [<Endereco: Feira Marina Barbosa, 65>, <Endereco: Estrada Rebeca Castro, 102>]>,
 <Pessoa: None, 715.038.429-32, Paulo, Endereço: [<Endereco: Viaduto Benício Ferreira, 1>, <Endereco: Rodovia de Farias, 6>]>,
 <Pessoa: None, 361.479.250-16, Yuri, Endereço: [<Endereco: Estação de Pires, 13>]>,
 <Pessoa: None, 381.067.294-78, Arthur, Endereço: [<Endereco: Pátio das Neves, 25>, <Endereco: Ladeira de Moreira, 70>, <Endereco: Chácara de Ramos, 9>]>,
 <Pessoa: None, 847.523.106-35, Antônio, Endereço: [<Endereco: Residencial Fernando Novaes, 69>, <Endereco:

In [8]:
pp.pprint(lista_pessoas[0].pk)

None


### Inserindo os documentos no banco de dados

In [9]:
# inserindo 1 documento apenas
lista_pessoas[0].save()

<Pessoa: 625ff866b7682e563753489e, 527.946.803-74, Rafael, Endereço: [<Endereco: Lago das Neves, 21>, <Endereco: Condomínio Ana Clara da Cunha, 11>]>

In [10]:
pp.pprint(lista_pessoas[0].pk)

ObjectId('625ff866b7682e563753489e')


In [11]:
# inserindo todos os outros documentos
Pessoa.objects.insert(lista_pessoas[1:])

[<Pessoa: 625ff866b7682e563753489f, 290.814.356-98, Yuri, Endereço: [<Endereco: Vale Moraes, 721>, <Endereco: Via de Dias, 114>, <Endereco: Largo Lima, 27>]>,
 <Pessoa: 625ff866b7682e56375348a0, 056.273.149-07, Maria Sophia, Endereço: [<Endereco: Vereda Azevedo, 875>, <Endereco: Vereda de Viana, 82>]>,
 <Pessoa: 625ff866b7682e56375348a1, 840.657.129-20, Lucas, Endereço: [<Endereco: Feira Marina Barbosa, 65>, <Endereco: Estrada Rebeca Castro, 102>]>,
 <Pessoa: 625ff866b7682e56375348a2, 715.038.429-32, Paulo, Endereço: [<Endereco: Viaduto Benício Ferreira, 1>, <Endereco: Rodovia de Farias, 6>]>,
 <Pessoa: 625ff866b7682e56375348a3, 361.479.250-16, Yuri, Endereço: [<Endereco: Estação de Pires, 13>]>,
 <Pessoa: 625ff866b7682e56375348a4, 381.067.294-78, Arthur, Endereço: [<Endereco: Pátio das Neves, 25>, <Endereco: Ladeira de Moreira, 70>, <Endereco: Chácara de Ramos, 9>]>,
 <Pessoa: 625ff866b7682e56375348a5, 847.523.106-35, Antônio, Endereço: [<Endereco: Residencial Fernando Novaes, 69>, <E

## Recuperando os documentos inseridos

#### Recuperar todos

In [12]:
for p in Pessoa.objects:
    pp.pprint(p.id)

ObjectId('625ff866b7682e563753489e')
ObjectId('625ff866b7682e563753489f')
ObjectId('625ff866b7682e56375348a0')
ObjectId('625ff866b7682e56375348a1')
ObjectId('625ff866b7682e56375348a2')
ObjectId('625ff866b7682e56375348a3')
ObjectId('625ff866b7682e56375348a4')
ObjectId('625ff866b7682e56375348a5')
ObjectId('625ff866b7682e56375348a6')
ObjectId('625ff866b7682e56375348a7')


In [13]:
pessoa = Pessoa.objects.first()
pp.pprint(pessoa)

<Pessoa: 625ff866b7682e563753489e, 527.946.803-74, Rafael, Endereço: [<Endereco: Lago das Neves, 21>, <Endereco: Condomínio Ana Clara da Cunha, 11>]>


In [14]:
pessoa.delete()

In [15]:
Pessoa.objects(id='625f4e6f1182b6d0d7a2dcb7')

[]